In [19]:
import numpy as np
import random
import matplotlib.pyplot as plt
import enum
import itertools

%matplotlib inline

In [31]:
@enum.unique
class Grade(enum.IntEnum):
    """
    Represents relevance on a graded scale. 
    Any positive number represents relevance, with higher numbers representing higher relevance (total ordering)
    Zero represents irrelevant
    """
    N = (0)   # Not relevant
    R = (1)   # Relevant
    HR = (2)  # Highly relevant

    
    @property
    def is_relevant(self):
        """
        'Binarizes' the relevance. Useful for computing binary metrics like precision / recall
        """
        return False if self.value == 0 else True
    
    @classmethod
    def from_int_list(cls, int_list):
        """
        Converts a list of integers to a list of Grade
        """
        members = dict((member.value, member) for member in cls.__members__.values())
        return [members[value] for value in int_list]

In [61]:
def get_all_permutations(grade, sequence_length):
    """
    For a given grade and a sequence length n, returns all possible permutations of length n that can be formed. 
    Note that a generator is returned and not a list
    """
    for i, perm in enumerate(itertools.product(range(len(grade.__members__)), repeat=sequence_length)):
        yield grade.from_int_list(perm)

def get_all_permutation_pairs(grade, sequence_length):
    """
    Generates all possible pairs of permutations given a grade and a sequence length.
    Note that a generator is returned and not a list
    """
    sequence_1 = get_all_permutations(grade, sequence_length)
    sequence_2 = get_all_permutations(grade, sequence_length)    
    for pair in itertools.product(sequence_1, sequence_2):
        p1, p2 = pair
        # don't return exact duplicates
        if all(map(lambda _ : _[0] == _[1], zip(p1, p2))) == True:
            continue
        yield pair

def count_sequence(seq):
    """
    `len` for generators
    """
    count = 0
    for _ in seq:
        count += 1
    return count

# A sanity check to ensure the correct number of sequences are being generated
# the `-(3**5)` removes exact duplicates
assert count_sequence(get_all_permutation_pairs(Grade, 5)) == (3**5) * (3**5) - (3 ** 5)

## Step 1: Simulate Rankings of Relevance for E and P (5 points)

In the first step you will generate pairs of rankings of relevance, for the production P and experimental E, respectively, for a hypothetical query q. Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all possible P and E ranking pairs of length 5. This step should give you about.

Example:

P: {N N N N N}
E: {N N N N R}

…
P: {HR HR HR HR R}
E: {HR HR HR HR HR}

In [62]:
permutations = list(get_all_permutation_pairs(Grade, 5))

In [63]:
permutations[0]

([<Grade.N: 0>, <Grade.N: 0>, <Grade.N: 0>, <Grade.N: 0>, <Grade.N: 0>],
 [<Grade.N: 0>, <Grade.N: 0>, <Grade.N: 0>, <Grade.N: 0>, <Grade.R: 1>])

In [64]:
permutations[-1]

([<Grade.HR: 2>, <Grade.HR: 2>, <Grade.HR: 2>, <Grade.HR: 2>, <Grade.HR: 2>],
 [<Grade.HR: 2>, <Grade.HR: 2>, <Grade.HR: 2>, <Grade.HR: 2>, <Grade.R: 1>])

## Step 2: Implement Evaluation Measures (10 points)
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 

In [5]:
class Metric:
    def compute(production, experimental, **kwargs):
        raise NotImplementedError()

class ContigencyTable(Metric):
    def __init__(self):
        self.true_positive = None
        self.true_negative = None
        self.false_negative = None
        self.false_positive = None
    
    def compute(production, experimental):
        for p, e in zip(production, experimental):
            # binarize
            p, e = p.is_relevant, e.is_relevant
            if p == True and e == True:
                self.true_positive += 1
            elif p == True and e == False:
                self.false_negative += 1
            elif p == False and e == True:
                self.false_positive += 1
            else:
                self.false_negative += 1    
        
        
class Precision(Metric):
    
    @property
    def name(self):
        return "precision"
    
    def compute(production, experimental, **kwargs):
        if k not in kwargs:
            raise ValueError("Precision requires parameter 'k'")

class Recall(Metric):
    
    @property
    def name(self):
        return "recall"
    
    
    def compute(production, experimental, **kwargs):
        if k not in kwargs:
            raise ValueError("Recall requires parameter 'k'")

class AveragePrecision(Metric):
    def compute(production, experimental, **kwargs):
        if k not in kwargs:
            raise ValueError("Recall requires parameter 'k'")


class GradedMetric:
    def compute(production, experimental, grade, **kwargs):
        raise NotImplementedError()
    
class NDCG(GradedMetric):
    pass

class ERR(GradedMetric):
    pass


## Step 3: Calculate the 𝛥measure (0 points)

For the three measures and all P and E ranking pairs constructed above calculate the difference: 𝛥measure = measureE-measureP. Consider only those pairs for which E outperforms P.

In [ ]:
del_measures_binary = {
    "precision": np.zeros(len(permutations)),
    "recall": np.zeros(len(permutations)),
}

del_measures_graded = {
    "EER": np.zeros(len(permutations)),
    "NDCG": np.zeros(len(permutations))
}